<H1>Resume Parser</H1>
<img src="https://cdn-dcklf.nitrocdn.com/dvFWjbaNIYCTKBUPoLoJwuqwNdymMqps/assets/static/optimized/rev-4efc3e8/wp-content/uploads/2018/08/1513589398.jpg" alt="Resume"
 width="500" 
height="300">

Dataset information: https://www.kaggle.com/datasets/dataturks/resume-entities-for-ner

Goal: Apply NER Model to parse a given resume 

<h1> Import Dataset </h1>

In [ ]:
#importing Google Drive 
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('/content/drive/My Drive/Resume_NER.json', lines=True)

In [ ]:
df.head()

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [ ]:
df.tail()

,content,annotation,extras
215,"Mansi Thanki\nStudent\n\nJamnagar, Gujarat - E...","[{'label': ['College Name'], 'points': [{'star...",NaN
216,Anil Kumar\nMicrosoft Azure (Basic Management)...,"[{'label': ['Location'], 'points': [{'start': ...",NaN
217,Siddharth Choudhary\nMicrosoft Office Suite - ...,"[{'label': ['Skills'], 'points': [{'start': 78...",NaN
218,Valarmathi Dhandapani\nInvestment Banking Oper...,"[{'label': ['Skills'], 'points': [{'start': 92...",NaN
219,Pradeep Kumar\nSecurity Analyst in Infosys - C...,"[{'label': ['Skills'], 'points': [{'start': 58...",NaN


<h1>Preprocess Data</h1>

In [ ]:
#A function to clean the dataset 
def cleanText(input):
   df = input.replace('\n',' ', regex=True) #Replace new lines with space
   df = df.drop('extras', axis=1) #Drop the redundant column 
   df = df.sample(frac=1).reset_index(drop=True) #Shuffing the dataset index for trials 
   return df 

In [ ]:
df = cleanText(df)

In [ ]:
df['content']

0      Debasish Dasgupta Trainer-Finacle-Core Banking...
1      Pratik Vaidya Pune, Maharashtra - Email me on ...
2      Manisha Bharti Software Automation Engineer  P...
3      kimaya sonawane Thane, Maharashtra - Email me ...
4      Alok Khandai Operational Analyst (SQL DBA) Eng...
                             ...                        
215    Siddharth Choudhary Microsoft Office Suite - E...
216    Mohini Gupta Server Support Engineer  Gurgaon,...
217    Vikas Singh Chandigarh, Chandigarh - Email me ...
218    Hartej Kathuria Data Analyst Intern - Oracle R...
219    Kiran Kumar I Having 2.1 years of Experience i...
Name: content, Length: 220, dtype: object

In [ ]:
df.isnull().values.any()

False

In [ ]:
print(f'The dataset has a total of {len(df.index)} resumes')

The dataset has a total of 220 resumes


In [ ]:
#Printing the first resume in dataframe
df['content'][0]

'Debasish Dasgupta Trainer-Finacle-Core Banking Solutions-Infosys - Onward eServices limited  Qasba, Bihar - Email me on Indeed: indeed.com/r/Debasish-Dasgupta/a20561e10f83ae3f  ✓ Worked as a faculty for Infosys in PTC ( Postal Training Centre) of DOP (Department of Post- India) ✓ Undertaken Classes for Banking Theory and practices. ✓ Trainer- Finacle Core Banking Solutions ✓ Maintaining client base of 20 Crores. ✓ Investment Banking ( H.N.I section) ✓ Admin and ATM manager (IT and branch) ✓ Management and smooth functioning of ATM network and administration. ✓ Implementation of Finacle10.2 in the branch level (Axis Bank) ✓ Successfully completed the task of Transition Supervisor in the migration from Finacle7 to Finacle 10.2. ✓ Over 4 years of rich experience in financial sector ✓ Ability to support and sustain a positive work environment that fosters team performance with strong communication and negotiation skills. ✓ Thorough understanding of cash management services involving bulk 

In [ ]:
!pip install spacy

In [ ]:
def preprocessData(inputData):
  #Restructuring the dataframe in spacy format
  entities = []
  for i in range(len(inputData.index)):
    
    toLabel = inputData['annotation'][i]
    elist = []
    
    for j in range(len(toLabel)):
    
      label = toLabel[j]['label']
      if len(label) > 0:
        label = label[0]
      start = toLabel[j]['points'][0]['start']
      end = toLabel[j]['points'][0]['end']
      elist = [*elist, tuple([start,end,label])]  
      edict = {'entities':elist}
    
    entities.append(edict)
  
  return entities 

In [ ]:
entities = preprocessData(df)

In [ ]:
sentences = df['content'].tolist()

In [ ]:
#A portion of the dataframe after conversion 
entities[1]

{'entities': [(1901, 1905, 'Skills'),
  (1812, 1815, 'Graduation Year'),
  (1763, 1797, 'College Name'),
  (1756, 1760, 'Degree'),
  (803, 806, 'Graduation Year'),
  (653, 656, 'Graduation Year'),
  (645, 648, 'Graduation Year'),
  (469, 478, 'Companies worked at'),
  (461, 466, 'Designation'),
  (403, 406, 'Location'),
  (54, 97, 'Email Address'),
  (14, 17, 'Location'),
  (0, 12, 'Name')]}

<h1> Training the model </h1> 

In [ ]:
import spacy
nlp = spacy.blank('en')
def NER_model(sentences,entities,n): 
 
  #Creating the built-in pipline components 
  if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
 
 
  #Adding labels 
  for eachEntity in entities:
    elist = eachEntity['entities']
    for ent in range(len(elist)):
      label = elist[ent][2]
      label = str(label)
      ner.add_label(label)
 
  #Disable other pipes in training 
  other_pipelines = [pipe for pipe in nlp.pipe_names if pipe!='ner']
 
  with nlp.disable_pipes(*other_pipelines):
    #Train the model 
    optimizer = nlp.begin_training()
    for i in range(n):
      print(f"Iteration {i}:")
      losses = {}
      temp = 0

      for temp in range(len(sentences)):
        try:
          nlp.update([sentences[temp]], [entities[temp]], drop = 0.2, sgd = optimizer, losses = losses )
        except Exception as e:
          pass
      print('Loss: ' +str(losses))

In [ ]:
trainedData = NER_model(sentences,entities,10) 

Iteration 0:
Loss: {'ner': 10771.07683084369}
Iteration 1:
Loss: {'ner': 12394.833622767888}
Iteration 2:
Loss: {'ner': 13346.55944420256}
Iteration 3:
Loss: {'ner': 10055.561846235418}
Iteration 4:
Loss: {'ner': 10536.417472127132}
Iteration 5:
Loss: {'ner': 9268.727689689495}
Iteration 6:
Loss: {'ner': 7263.374293092989}
Iteration 7:
Loss: {'ner': 6481.302725220139}
Iteration 8:
Loss: {'ner': 8506.186095533074}
Iteration 9:
Loss: {'ner': 8805.119950339247}


In [ ]:
#Saving and deploying the NER model 
nlp.to_disk('ner_model')
ner_model = spacy.load('ner_model')
print("The Model is now ready to use")

The Model is now ready to use


In [ ]:
#Taking a look at the labels acknowledged by the NER model 
def extractEntities(model):
 enfound = list(model)
 enfound = [ele for ele in enfound if ele != '[]' if ele!='UNKNOWN']
 return enfound

In [ ]:
size = len(extractEntities(ner_model.entity.labels))
print(f'There were {size} entities found by the NER model, as follows: \n')
extractEntities(ner_model.entity.labels)

There were 10 entities found by the NER model, as follows: 



['College Name',
 'Companies worked at',
 'Degree',
 'Designation',
 'Email Address',
 'Graduation Year',
 'Location',
 'Name',
 'Skills',
 'Years of Experience']

<h1> Testing the Model </h1>

In [ ]:
import random 
#Randomly selecting one of the resumes in the dataframe 
index = random.randint(0, 220)
testData = df['content'][index]

In [ ]:
testData 

'Rajeev Kumar Senior Associate Consultant - Infosys/Offshore Lead  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Rajeev-Kumar/3f560fd91275495b  •An erudite professional with 2 years of IT/Presales/Project Management experience •Possess knowledge of Project Management, Digital/Social Media Marketing, Analytics, BI (Business Intelligence), presales, Business development, strategy and planning, etc. •Endowed with a passion for winning as exemplified through excellence in academic, extracurricular areas and organizational experience •An effective communicator with strong analytical/logical skills to relate to people at any level of business and management •Exposure in improving business performance by leveraging market insights to influence strategic and tactical plans that delivered differentiated offerings to customers •Possess problem solving capability keeping constraints in purview, innovation &amp; adaptability •A quick learner with the expertise to work under pressure and 

In [ ]:
#Testing the above selected resume 
doc = ner_model(testData)
for entity in doc.ents:
  print(f"{entity.label_.capitalize():{15}}:    {entity.text}")

Degree         :    PGDM in Marketing
College name   :    Goa Institute
Degree         :    B.Tech in Computer Science and Engineering
Skills         :    Adobe Photoshop (Less than 1 year), Ads (Less than 1 year), BI (1 year), Business Intelligence (1 year), MS Excel (2 years), presales (1 year), Tableau (1 year), Pega Marketing and BPM (Less than 1 year), MS Word (2 years), MS Powerpoint (2 years), MS Sharepoint (2 years), Spss (Less than 1 year), IBM Maximo (Less than 1 year), marketing strategy (2 years), Business Analysis (2 years), communication and soft skills (2 years), Project Management (2 years), Project Planning (2 years), Team Management (1 year)  LINKS  http://www.linkedin.com/in/rajeevkumar91  CERTIFICATIONS/LICENSES  Google Adwords  December 2016 to December 2017  Google Analytics  February 2017 to August 2018  HubSpot Inbound Marketing  Digital Vidya Certified Digital Marketer  http://www.linkedin.com/in/rajeevkumar91   ADDITIONAL INFORMATION  Soft Skills  Communicator

In [ ]:
!pip install PyMUpdf

     |████████████████████████████████| 8.8 MB 6.2 MB/s 


In [ ]:
!pip install python-docx 

     |████████████████████████████████| 5.6 MB 8.3 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=08b4e5f8cb465ee4328f39cdf4a8aedb860ec468cfbbb5896d8c35b220ba1993
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [ ]:
import glob 
import fitz 
import docx 

#A function to recieve an input Resume in .pdf or .docx format
def readResume(path):
  if path.endswith('.pdf'):
    resume = fitz.open(path)
    text = ""
    for page in resume:
      text = text + str(page.get_text())
    testResume = " ".join(text.split('\n'))
    return testResume

  elif path.endswith('.docx'):
    resume = docx.Document(path)
    text = ""
    for page in resume.paragraphs:
        text = text + page.text
    testResume = " ".join(text.split('\n'))
    return testResume

  else: 
    print('No other file extensions supported')

In [ ]:
testResume = readResume('/content/drive/My Drive/Test Resume2.pdf')

In [ ]:
#Testing the Resume as fetched above
doc = ner_model(testResume)
for entity in doc.ents:
 print(f"{entity.label_.capitalize():{15}}:    {entity.text}")

Designation    :    Software Engineer
Degree         :    Indian Institute of Technology – Mumbai  2001    SKILLS
Skills         :    Machine Learning, Natural Language Processing, and Big Data Handling    ADDITIONAL INFORMATION  Professional Skills  • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal  skills with ability to interact with individuals at all the levels  • Quick learner and maintains cordial relationship with project manager and team members and  good performer both in team and independent job environments  • Positive attitude towards superiors &amp; peers  • Supervised junior developers throughout project lifecycle and provided technical assistance
